<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From 497a841a4020bb9ec7b278bf670b5d3b91b5e077 Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Tue, 6 Jul 2021 17:07:56 +0800
Subject: [PATCH] mindfl modify adbert to emotion classify demo

---
 .../java/com/mindspore/flclient/model/AdBert.java    | 20 +++++++++-----------
 .../mindspore/flclient/model/CustomTokenizer.java    | 11 +++++------
 .../java/com/mindspore/flclient/model/DataSet.java   |  5 +----
 3 files changed, 15 insertions(+), 21 deletions(-)

diff --git a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/AdBert.java b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/AdBert.java
index c53c504..fa28980 100644
--- a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/AdBert.java
+++ b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/AdBert.java
@@ -28,7 +28,7 @@ import java.util.logging.Logger;
 public class AdBert extends TrainModel {
     private static final Logger logger = Logger.getLogger(AdBert.class.toString());
 
-    private static final int NUM_OF_CLASS = 5;
+    private static final int NUM_OF_CLASS = 4;
 
     List<Feature> features;
 
@@ -66,7 +66,7 @@ public class AdBert extends TrainModel {
         tokenIdBufffer.order(ByteOrder.nativeOrder());
         maskIdBufffer.order(ByteOrder.nativeOrder());
         if (trainMod) {
-            labelIdBufffer = ByteBuffer.allocateDirect(inputSize * Integer.BYTES);
+            labelIdBufffer = ByteBuffer.allocateDirect(batchSize * Integer.BYTES);
             labelIdBufffer.order(ByteOrder.nativeOrder());
         }
         numOfClass = NUM_OF_CLASS;
@@ -93,14 +93,12 @@ public class AdBert extends TrainModel {
             for (int j = 0; j < dataSize; j++) {
                 maskIdBufffer.putInt(feature.inputMasks[j]);
             }
+            if(trainMod) {
+                labelIdBufffer.putInt(feature.labelIds);
+            }
             if (!trainMod) {
                 labels.add(feature.labelIds);
             }
-            if (trainMod) {
-                for (int j = 0; j < dataSize; j++) {
-                    labelIdBufffer.putInt(feature.inputIds[j]);
-                }
-            }
         }
 
         List<MSTensor> inputs = trainSession.getInputs();
@@ -109,10 +107,10 @@ public class AdBert extends TrainModel {
         MSTensor inputIdTensor;
         MSTensor maskIdTensor;
         if (trainMod) {
-            labelIdTensor = inputs.get(0);
-            tokenIdTensor = inputs.get(1);
-            inputIdTensor = inputs.get(2);
-            maskIdTensor = inputs.get(3);
+            labelIdTensor = inputs.get(3);
+            tokenIdTensor = inputs.get(0);
+            inputIdTensor = inputs.get(1);
+            maskIdTensor = inputs.get(2);
             labelIdTensor.setData(labelIdBufffer);
         } else {
             tokenIdTensor = inputs.get(0);
diff --git a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/CustomTokenizer.java b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/CustomTokenizer.java
index e986885..539bbe8 100644
--- a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/CustomTokenizer.java
+++ b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/CustomTokenizer.java
@@ -33,14 +33,13 @@ public class CustomTokenizer {
     private int maxInputChars = 100;
     private String[] NotSplitStrs = {"UNK"};
     private String unkToken = "[UNK]";
-    private int maxSeqLen = 16;
+    private int maxSeqLen = 8;
     private int vocabSize = 11682;
     private Map<String, Integer> labelMap = new HashMap<String, Integer>() {{
-        put("beauty", 0);
-        put("education", 1);
-        put("hotel", 2);
-        put("travel", 3);
-        put("other", 4);
+        put("good", 0);
+        put("leimu", 1);
+        put("xiaoku", 2);
+        put("xin", 3);
     }};
 
     public void init(String vocabFile, String idsFile, boolean trainMod, boolean doLowerCase) {
diff --git a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/DataSet.java b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/DataSet.java
index 5d75207..1c2f982 100644
--- a/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/DataSet.java
+++ b/mindspore/lite/java/java/fl_client/src/main/java/com/mindspore/flclient/model/DataSet.java
@@ -62,11 +62,8 @@ public class DataSet {
 
         List<Feature> features = new ArrayList<>(examples.size());
         for (int i = 0; i < examples.size(); i++) {
-            List<Integer> tokens = customTokenizer.tokenize(examples.get(i), trainMod);
+            List<Integer> tokens = customTokenizer.tokenize(examples.get(i), false);
             Feature feature = customTokenizer.getFeatures(tokens, labels.get(i));
-            if (trainMod) {
-                customTokenizer.addRandomMaskAndReplace(feature, true, true);
-            }
             features.add(feature);
         }
         return features;
-- 
2.7.4

